In [107]:
import requests
import pandas as pd
import numpy as np
import json
from bs4 import BeautifulSoup
import config
import logging
import importlib
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
import validators
import pickle
import time
import random
from calendar import monthrange
import re
import spacy
from spacy.tokens import Doc, Span
import nltk
from nltk.corpus import stopwords
from nltk.corpus import treebank
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import sent_tokenize
from nltk import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import BaseEstimator, TransformerMixin

from nltk import pos_tag, sent_tokenize, wordpunct_tokenize
from nltk.corpus import wordnet

pd.options.display.max_columns = 300
import string
import unicodedata

pd.options.display.max_columns = 200
pd.options.display.max_rows = 999
import matplotlib.pyplot as plt  

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.datasets as datasets
import pandas as pd
import numpy as np
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
from sklearn.metrics import mean_squared_error
from pandas.plotting import scatter_matrix
import seaborn as sns
import scipy.stats as stats
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.dummy import DummyClassifier
from sklearn.externals import joblib
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn import tree 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.externals.six import StringIO  
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
import xgboost as xgb
from sklearn.svm import SVC  
from time import time
np.random.seed(0)
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from sklearn.naive_bayes import MultinomialNB
from sklearn.decomposition import LatentDirichletAllocation

import gensim
from gensim import corpora
from gensim import models
from gensim.sklearn_api import lsimodel, ldamodel
from gensim.matutils import sparse2full
from gensim.corpora import Dictionary
from gensim.models.tfidfmodel import TfidfModel

## Helper Class/Function for cleaning

In [145]:
class TextNormalizer(BaseEstimator, TransformerMixin): 
    def __init__(self, language='english'):
        self.stopwords = set(nltk.corpus.stopwords.words(language))
        self.lemmatizer = WordNetLemmatizer()
    
    def is_punct(self, token):
        return all(unicodedata.category(char).startswith('P') for char in token)
    
    def is_stopword(self, token):
        return token.lower() in self.stopwords
    
    def get_wordnet_pos(selfm, word):
        """Map POS tag to first character lemmatize() accepts"""
        tag = nltk.pos_tag([word])[0][1][0].upper()
        tag_dict = {"J": wordnet.ADJ,
                    "N": wordnet.NOUN,
                    "V": wordnet.VERB,
                    "R": wordnet.ADV}

        return tag_dict.get(tag, wordnet.NOUN)   
    
    def normalize(self, document, source):
        NYT_PAT = '^[A-Z]* — '
        NYT_COMP_PAT = re.compile(NYT_PAT)
        norm_toks = []
        
        if source == 'NYT':        
            document = NYT_COMP_PAT.sub('', document)
            
        for sent in sent_tokenize(document):
            for tok in nltk.word_tokenize(sent):
                if self.is_stopword(tok) or self.is_punct(tok) or tok.isdigit():
                    continue
                
                norm_toks.append(self.lemmatize(tok, get_wordnet_pos(tok)).lower())        
        return norm_toks

                
    def lemmatize(self, token, pos):
        return self.lemmatizer.lemmatize(token, pos)
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, documents):
        for doc in documents:
            yield self.normalize(document)
        

## Load and add to Stop Words

In [64]:
my_stopwords = set(stopwords.words('english'))

In [65]:
my_stopwords.add('ms')

## Load NYT articles first for pre-processing

In [146]:
nyt_df = pd.DataFrame.from_csv('ALL_NYT_WORLD.csv')

In [147]:
nyt_df.head()

,date,headline,paper,section,text,url
0,2019-10-01,Finland School Stabbing Leaves 1 Dead and 10 I...,NYT,World,A stabbing attack on Tuesday at a vocational c...,https://www.nytimes.com/2019/10/01/world/europ...
2,2019-10-01,"Protests Erupt in Hong Kong, Overshadowing Chi...",NYT,World,BEIJING — China’s authoritarian president used...,https://www.nytimes.com/2019/10/01/world/asia/...
4,2019-10-02,China’s Global Message: We Are Tough but Not T...,NYT,World,BEIJING — As President Xi Jinping watched from...,https://www.nytimes.com/2019/10/02/world/asia/...
6,2019-10-02,Putin Says He Doesn’t Mind if Private Talk Wit...,NYT,World,MOSCOW — President Vladimir V. Putin of Russia...,https://www.nytimes.com/2019/10/02/world/europ...
11,2019-10-03,South Africa Vows to Crack Down on Violence Ag...,NYT,World,"PRETORIA, South Africa — The president of Sout...",https://www.nytimes.com/2019/10/03/world/afric...


In [148]:
len(nyt_df)

2379

In [149]:
nyt_df.dropna(inplace=True)

In [150]:
text_normalizer = TextNormalizer()

In [96]:
from functools import partial

In [151]:
txfm = partial(text_normalizer.normalize, source='NYT')

In [152]:
nyt_df['norm_text'] = nyt_df.text.apply(txfm)

In [153]:
nyt_df.head(10)

,date,headline,paper,section,text,url,norm_text
0,2019-10-01,Finland School Stabbing Leaves 1 Dead and 10 I...,NYT,World,A stabbing attack on Tuesday at a vocational c...,https://www.nytimes.com/2019/10/01/world/europ...,"[stab, attack, tuesday, vocational, college, s..."
2,2019-10-01,"Protests Erupt in Hong Kong, Overshadowing Chi...",NYT,World,BEIJING — China’s authoritarian president used...,https://www.nytimes.com/2019/10/01/world/asia/...,"[china, authoritarian, president, use, 70th, a..."
4,2019-10-02,China’s Global Message: We Are Tough but Not T...,NYT,World,BEIJING — As President Xi Jinping watched from...,https://www.nytimes.com/2019/10/02/world/asia/...,"[president, xi, jinping, watch, rostrum, iconi..."
6,2019-10-02,Putin Says He Doesn’t Mind if Private Talk Wit...,NYT,World,MOSCOW — President Vladimir V. Putin of Russia...,https://www.nytimes.com/2019/10/02/world/europ...,"[president, vladimir, v., putin, russia, say, ..."
11,2019-10-03,South Africa Vows to Crack Down on Violence Ag...,NYT,World,"PRETORIA, South Africa — The president of Sout...",https://www.nytimes.com/2019/10/03/world/afric...,"[pretoria, south, africa, president, south, af..."
14,2019-10-03,Boris Johnson’s Brexit Plan Hits a Wall in Bru...,NYT,World,"LONDON — On Thursday, Prime Minister Boris Joh...",https://www.nytimes.com/2019/10/03/world/europ...,"[thursday, prime, minister, boris, johnson, ap..."
20,2019-10-03,"Hong Kong Police, Seen as ‘Hounds After Rabbit...",NYT,World,HONG KONG — A woman was hit in the eye during ...,https://www.nytimes.com/2019/10/03/world/asia/...,"[hong, kong, woman, hit, eye, protest, passeng..."
21,2019-10-03,Signs of Ballot-Box Stuffing Add Tension to Af...,NYT,World,"KABUL, Afghanistan — By early afternoon on Ele...",https://www.nytimes.com/2019/10/03/world/asia/...,"[kabul, afghanistan, early, afternoon, electio..."
23,2019-10-01,"Hours After Agreeing to Resume Talks, North Ko...",NYT,World,"SEOUL, South Korea — North Korea launched at l...",https://www.nytimes.com/2019/10/01/world/asia/...,"[seoul, south, korea, north, korea, launch, le..."
28,2019-10-02,"To Disrupt Elections, Taliban Turn to an Old T...",NYT,World,"KABUL, Afghanistan — There are several reasons...",https://www.nytimes.com/2019/10/02/world/asia/...,"[kabul, afghanistan, several, reason, afghan, ..."


In [106]:
nyt_df[['norm_text']]

,norm_text
0,stab attack tuesday vocational college shoppin...
2,china authoritarian president use 70th anniver...
4,president xi jinping watch rostrum iconic port...
6,president vladimir v. putin russia say wednesd...
11,pretoria south africa president south africa v...
14,thursday prime minister boris johnson appear a...
20,hong kong woman hit eye protest passengers bea...
21,kabul afghanistan early afternoon election day...
23,seoul south korea north korea launch least one...
28,kabul afghanistan several reason afghan offici...


In [155]:
lexicon = Dictionary([[word for word in doc] for doc in nyt_df.norm_text])

In [160]:
corpus = [lexicon.doc2bow(doc) for doc in nyt_df.norm_text]

In [139]:
corpus

[[], [], [], [], [(38, 1)], [(22, 1)], [], [(22, 1)], [], [(22, 1), (38, 2)], [], [], [(22, 1)], [], [(22, 2)], [], [], [], [], [(38, 1)], [(22, 2)], [], [], [], [], [(22, 1)], [(22, 2)], [], [], [], [(22, 1)], [], [(38, 2)], [], [(38, 1)], [], [], [(38, 1)], [(38, 1)], [(38, 1)], [], [], [], [(22, 1)], [], [], [], [], [(22, 1), (38, 1)], [(22, 2), (38, 1)], [(38, 1)], [], [(22, 1)], [], [], [(22, 1), (38, 2)], [(38, 1)], [], [(38, 3)], [], [], [], [], [(22, 2)], [], [(22, 1), (38, 1)], [], [], [], [(38, 1)], [], [], [], [(38, 1)], [(22, 1)], [], [], [(22, 2)], [], [], [(22, 1)], [(38, 3)], [], [], [], [], [(38, 5)], [], [], [], [], [(22, 1)], [], [], [(22, 1)], [], [], [(35, 1)], [], [], [], [], [(22, 1)], [], [(22, 1), (38, 1)], [(38, 1)], [(38, 1)], [(22, 2), (38, 2)], [], [(38, 3)], [(38, 1)], [(38, 1)], [(38, 1)], [], [(22, 4)], [], [], [], [(38, 1)], [(38, 8)], [(22, 2)], [(38, 1)], [], [(22, 4)], [], [], [], [], [(38, 1)], [], [], [], [(22, 3)], [(38, 1)], [], [(22, 2)], [(23, 1

In [161]:
tfidf = TfidfModel([lexicon.doc2bow(doc) for doc in nyt_df.norm_text], id2word=lexicon)

In [167]:
model = models.LdaModel(corpus, id2word=lexicon, num_topics=25)

In [168]:

# # show the topics with the token-weights for the top 10 most influential tokens:
print(model.print_topics(10))

[(1, '0.020*"mr." + 0.018*"say" + 0.007*"party" + 0.006*"european" + 0.006*"government" + 0.005*"would" + 0.004*"trump" + 0.004*"election" + 0.004*"new" + 0.004*"year"'), (24, '0.011*"say" + 0.006*"mr." + 0.006*"american" + 0.006*"official" + 0.005*"force" + 0.005*"special" + 0.004*"islamic" + 0.004*"military" + 0.003*"state" + 0.003*"two"'), (3, '0.019*"say" + 0.007*"people" + 0.006*"mr." + 0.006*"hong" + 0.006*"government" + 0.005*"kong" + 0.005*"would" + 0.005*"protester" + 0.004*"official" + 0.004*"protest"'), (16, '0.019*"mr." + 0.016*"say" + 0.007*"would" + 0.006*"party" + 0.005*"one" + 0.004*"new" + 0.004*"police" + 0.004*"take" + 0.004*"year" + 0.004*"people"'), (13, '0.026*"mr." + 0.012*"say" + 0.012*"trump" + 0.006*"taiwan" + 0.006*"president" + 0.005*"china" + 0.005*"country" + 0.004*"party" + 0.004*"would" + 0.004*"year"'), (0, '0.022*"hong" + 0.018*"kong" + 0.018*"say" + 0.012*"police" + 0.011*"china" + 0.010*"mr." + 0.010*"protester" + 0.009*"bill" + 0.009*"lam" + 0.009*"

In [169]:
#     corpus = [
#         gmodel.model.named_steps['vect'].lexicon.doc2bow(doc)
#         for doc in gmodel.model.named_steps['norm'].transform(docs)
#     ]
    

id2token = lexicon.id2token

for word_id, freq in next(iter(corpus)):
    print(id2token[word_id], freq)

# get the highest weighted topic for each of the documents in the corpus
def get_topics(vectorized_corpus, model):
    from operator import itemgetter

    topics = [
        max(model[doc], key=itemgetter(1))[0]
        for doc in vectorized_corpus
    ]

    return topics

topics = get_topics(corpus, model)

for topic, doc in zip(topics, nyt_df.norm_text):
    print("Topic:{}".format(topic))
    print(doc)

accord 1
add 1
antti 1
area 2
arrest 2
assailant 1
attack 3
bladed 1
bleeding 1
blomberg 1
blood 1
call 1
center 2
city 1
class 1
college 3
come 1
completely 1
evacuate 1
finland 1
finnish 3
firearm 2
floor 1
government 1
heard 1
helsingin 1
herman 1
hospital. 1
injured 4
injury 1
investigation 1
janette 1
kill 1
kuopio 2
leave 1
local 1
man 1
member 1
minister 1
minor 1
monitoring 1
mtv 1
nearby 1
news 1
newspaper 1
noon 1
officer 1
one 1
others 1
outlet 1
people.another 1
person 1
police 4
police.reports 1
possession 1
reportedly 1
rinne 1
sabre-like 1
sanomat 1
savo 1
saw 1
say 5
school 1
seriously 1
several 1
shock 1
shopping 2
shortly 1
shout 1
situation 1
stab 1
staff 1
statement 1
student 4
suffer 1
suspect 2
sword.prime 1
teacher 1
time 1
told 2
treat 1
tuesday 1
unacceptable 1
underway 1
university 1
upon 1
use 1
violence 1
vocational 2
weapon 1
well 1
witness 1
wound 1


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Topic:1
['four', 'year', 'ago', 'almost', 'day', 'greece', 'come', 'breath', 'away', 'leave', 'euro', 'people', 'form', 'quiet', 'somber', 'line', 'outside', 'bank', 'take', 'small', 'amount', 'cash', 'lockdown', 'financial', 'system', 'bar', 'access', 'saving', 'stockpile', 'tin', 'food', 'toilet', 'paper.on', 'sunday', 'leader', 'rally', 'greeks', 'confront', 'international', 'creditor', 'reverse', 'give', 'demand', 'austerity', 'vote', 'office', 'prime', 'minister', 'alexis', 'tsipras', 'former', 'firebrand', 'leftist', 'defeat', 'landslide.greeks', 'turn', 'instead', 'resurgent', 'center-right', 'new', 'democracy', 'party', 'lead', 'kyriakos', 'mitsotakis', 'harvard-educated', 'former', 'banker', 'son', '1990s', 'prime', 'minister', 'party', 'secure', 'almost', 'percent', 'vote', 'comfortable', 'majority', 'seat', '300-seat', 'parliament.mr', 'tsipras', 'party', 'syriza', 'get', 'percent', 'vote', 'sting', 'defeat', 'prime', 'minister', 'personally', 'bookend', 'begin', 'social', '

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
model.

In [140]:
import pyLDAvis
import pyLDAvis.gensim

In [ ]:
data = pyLDAvis.gensim.prepare(model, corpus, lexicon)

In [144]:
pyLDAvis.save_html(data, 'gensim.html')

TypeError: Object of type complex is not JSON serializable